In [32]:
import pickle 
import pandas as pd 
import numpy as np
from scipy import stats
import plotly_express as px

### LDA topic modeling 
These are the topics in our corpus. 

In [33]:
with open (r'data/df_best_category.pkl', 'rb') as input_file: 
    df_best_category = pickle.load(input_file)

In [34]:
with open (r"data/lda_model_1.pkl", "rb") as input_file:
    lda_model = pickle.load(input_file)

In [35]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.007*"century" + 0.007*"literature" + 0.006*"write" + 0.004*"book" + 0.004*"displaystyle" + 0.004*"bear" + 0.003*"ancient" + 0.003*"roman" + 0.003*"form" + 0.003*"museum"
Topic: 1 
Words: 0.006*"self" + 0.006*"music" + 0.006*"culture" + 0.006*"people" + 0.005*"health" + 0.004*"psychology" + 0.004*"group" + 0.003*"study" + 0.003*"social" + 0.003*"form"
Topic: 2 
Words: 0.007*"social" + 0.006*"study" + 0.005*"theory" + 0.004*"science" + 0.004*"human" + 0.004*"philosophy" + 0.004*"self" + 0.004*"research" + 0.003*"society" + 0.003*"people"
Topic: 3 
Words: 0.006*"film" + 0.006*"series" + 0.005*"star" + 0.004*"television" + 0.004*"american" + 0.003*"award" + 0.003*"music" + 0.003*"show" + 0.003*"release" + 0.003*"season"
Topic: 4 
Words: 0.009*"university" + 0.007*"program" + 0.007*"engineer" + 0.005*"school" + 0.005*"technology" + 0.005*"science" + 0.004*"college" + 0.004*"education" + 0.004*"research" + 0.004*"institute"
Topic: 5 
Words: 0.008*"food" + 0.006*"company" +

In [36]:
df_best_category['category'] = df_best_category['category'].replace({0:'0-literature', 1:'1-psychology'
                                                                     , 2:'2-sociology', 3:'3-film', 4:'4-STEM'
                                                                     , 5: '5-business', 6:'6-global'
                                                                     , 7:'7-language', 8:'8-cities', 9:'9-music'})

In [37]:
px.histogram(data_frame=df_best_category.sort_values('category')
     , x="category"
     , title="Topics In Corpus"
     , template='plotly'
     )